In [1]:
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types as genai_types

from builder import AgentBuilder

/home/yrlab/miniconda3/envs/agent/lib/python3.12/site-packages/pydantic/plugin/_schema_validator.py:39: UserWarning: ImportError while loading the `logfire-plugin` Pydantic plugin, this plugin will not be installed.

ImportError("cannot import name 'ReadableLogRecord' from 'opentelemetry.sdk._logs' (/home/yrlab/miniconda3/envs/agent/lib/python3.12/site-packages/opentelemetry/sdk/_logs/__init__.py)")
  plugins = get_plugins()


In [2]:
user_id="1234"
user_name="id4thomas"

In [3]:
agent = AgentBuilder.build(
    name="WeatherAgent",
    user_id=user_id,
    tool_names=["location", "weather"],
    user_name=user_name
)

In [4]:
query="내 현재 위치와 주변 도시들을 알려주고 그 도시들의 날씨들도 알려줘."
# query="서울 주변 도시들을 알려주고 그 도시들의 날씨들도 알려줘. 도시 명칭은 반환값 그대로 사용해"
# query="도구를 사용해서 서울 날씨 알려줘. 도구의 결과도 json 그대로 알려줘"


In [5]:
# Run
## Initialize Session
session_service = InMemorySessionService()
runner = Runner(
    agent=agent,
    app_name="WeatherApp",
    session_service=session_service,
)

In [6]:
## Run
session = await session_service.create_session(
    app_name="WeatherApp",
    user_id=user_id,
)
session_id = session.id

user_content = genai_types.Content(
    role="user",
    parts=[genai_types.Part(text=query)],
)

# Runner 실행 — 이벤트 스트림에서 최종 모델 응답 추출
final_response = ""
async for event in runner.run_async(
    user_id=user_id,
    session_id=session_id,
    new_message=user_content,
):
    # is_final_response()가 True인 이벤트의 텍스트가 최종 답변
    if event.is_final_response() and event.content:
        for part in event.content.parts:
            if hasattr(part, "text") and part.text:
                final_response += part.text

/home/yrlab/miniconda3/envs/agent/lib/python3.12/site-packages/google/adk/features/_feature_decorator.py:72: UserWarning: [EXPERIMENTAL] feature FeatureName.BASE_AUTHENTICATED_TOOL is enabled.
  check_feature_enabled()
/home/yrlab/miniconda3/envs/agent/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content=None, rol...one, 'reasoning': None}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='to...one, 'token_ids': None}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


/home/yrlab/miniconda3/envs/agent/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='현재 ...one, 'reasoning': None}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...one, 'token_ids': None}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


In [7]:
print(final_response)

현재 위치는 부산이며, 주변 도시로는 김해, 창원, 울산이 있습니다.

각 도시의 현재 날씨는 다음과 같습니다:
- 부산: 7°C, 구름 조금, 습도 62%, 바람 20km/h
- 김해: 6°C, 맑음, 습도 60%, 바람 10km/h
- 창원: 5°C, 구름 조금, 습도 54%, 바람 15km/h
- 울산: 6°C, 흐림, 습도 63%, 바람 14km/h


In [8]:
# Stream
## Initialize Session
session = await session_service.create_session(
    app_name="WeatherApp",
    user_id=user_id,
)
session_id = session.id

current_text = ""

In [ ]:
async for event in runner.run_async(
    user_id=user_id,
    session_id=session_id,
    new_message=user_content,
):
    print(f"[{event.author}]")
    # ── 1. 유저 이벤트 스킵 ──────────────────────────────
    if event.author == "user":
        continue

    # ── 2. Tool Call (LLM → Tool 요청) ──────────────────
    function_calls = event.get_function_calls()
    if function_calls:
        # 스트리밍 텍스트가 진행 중이었으면 줄바꿈
        if current_text:
            print()
            current_text = ""
        for call in function_calls:
            print(f"\n[tool_call] tool={call.name} args={call.args}")
        continue

    # ── 3. Tool Result (Tool → LLM 응답) ────────────────
    function_responses = event.get_function_responses()
    if function_responses:
        for resp in function_responses:
            print(f"[tool_result] tool={resp.name} result={resp.response}")
        continue

    # ── 4. 텍스트 스트리밍 / 완성 ──────────────────────
    if event.content and event.content.parts:
        for part in event.content.parts:
            if not hasattr(part, "text") or not part.text:
                continue

            if event.partial:
                # 스트리밍 청크: 이전 누적 텍스트 이후 델타만 출력
                delta = part.text[len(current_text):]
                current_text = part.text
                print(delta, end="", flush=True)
            else:
                # 완성된 텍스트
                if event.is_final_response():
                    # 스트리밍 중 final이 오면 남은 델타 처리
                    delta = part.text[len(current_text):]
                    if delta:
                        print(delta, end="", flush=True)
                    print()  # 줄바꿈
                    print(f"\n[final] {part.text}")
                    current_text = ""
                else:
                    # 중간 완성 텍스트 (non-partial, non-final)
                    print(part.text)

[WeatherAgent]

[tool_call] tool=get_user_location args={}
[WeatherAgent]
[tool_result] tool=get_user_location result={'content': [{'type': 'text', 'text': '{"name":"Seoul","coordinate":{"lat":37.5665,"lon":126.978}}'}], 'structuredContent': {'name': 'Seoul', 'coordinate': {'lat': 37.5665, 'lon': 126.978}}, 'isError': False}
[WeatherAgent]

[tool_call] tool=get_nearby_cities args={'city_name': 'Seoul'}
[WeatherAgent]
[tool_result] tool=get_nearby_cities result={'content': [{'type': 'text', 'text': '[{"name":"Seongnam","coordinate":{"lat":37.42,"lon":127.1267}},{"name":"Suwon","coordinate":{"lat":37.2636,"lon":127.0286}},{"name":"Changwon","coordinate":{"lat":35.228,"lon":128.6811}}]'}], 'structuredContent': {'result': [{'name': 'Seongnam', 'coordinate': {'lat': 37.42, 'lon': 127.1267}}, {'name': 'Suwon', 'coordinate': {'lat': 37.2636, 'lon': 127.0286}}, {'name': 'Changwon', 'coordinate': {'lat': 35.228, 'lon': 128.6811}}]}, 'isError': False}
[WeatherAgent]

[tool_call] tool=get_current

/home/yrlab/miniconda3/envs/agent/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='id4thoma...one, 'reasoning': None}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...one, 'token_ids': None}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
